In [119]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Dense, Bidirectional, LSTM, Dropout
import tensorflow as tf

In [18]:
import json
import random

In [4]:
datastore = [json.loads(line) for line in open('News_Category_Dataset_v2.json', 'r')]

In [60]:
category_embeddings = {"POLITICS": 0, "CRIME": 1, "ENTERTAINMENT": 2, "SPORTS": 3, "SCIENCE": 4}

In [61]:
short_description = []
category = []
urls = []
check = 0
random.shuffle(datastore)
max_len = 0
for item in datastore:
    if (item['category'] == 'POLITICS' or
       item['category'] == 'CRIME' or 
       item['category'] == 'ENTERTAINMENT' or 
       item['category'] == 'SPORTS' or 
       item['category'] == 'SCIENCE'):
        short_description.append(item['headline'])
        category.append(category_embeddings[item['category']])
        urls.append(item['link'])

In [62]:
print(len(short_description))

59264


In [81]:
vocab_size = 100000
training_size = 37000
max_length = 200
embedding_dim = 16

In [82]:
training_sentences = short_description[0:training_size]
testing_sentences = short_description[training_size:]
training_labels = category[0:training_size]
testing_labels = category[training_size:]

In [91]:
def get_max_len(sequence):
    max_len = 0
    for s in sequence: 
        max_len = max(max_len, len(s))
    return max_len

In [102]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
#maxlen_train = get_max_len(training_sequences)
#print(maxlen_train)
training_padded = pad_sequences(training_sequences, 
                                maxlen=33,
                                padding='post', 
                                truncating='post')

In [103]:
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
#maxlen_test = get_max_len(training_sequences)
#print(maxlen_test)
testing_padded = pad_sequences(testing_sequences, 
                                maxlen=33,
                                padding='post', 
                                truncating='post')

In [104]:
import numpy as np
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)

In [109]:
# Best Model          98% training, 87% testing
#model = tf.keras.Sequential([
#    Embedding(voc_size, 256),
#    LSTM(128, return_sequences=True, activation='relu'),
#    Dropout(0.2),
#    LSTM(128),
#    Dropout(0.2),
#    Dense(32, activation='relu'),
#    Dense(5, activation='sigmoid')
#])
#model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [126]:
# Best Model
model = tf.keras.Sequential([
    Embedding(voc_size, 256),
    LSTM(128, return_sequences=True, activation='relu'),
    Dropout(0.2),
    LSTM(128),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(5, activation='sigmoid')
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model = tf.

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [127]:
num_epochs = 10
history = model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose=2)

Epoch 1/10
1157/1157 - 244s - loss: 1.1588 - accuracy: 0.5652 - val_loss: 1.1723 - val_accuracy: 0.5539
Epoch 2/10
1157/1157 - 301s - loss: 1.1844 - accuracy: 0.5516 - val_loss: 1.1696 - val_accuracy: 0.5539
Epoch 3/10
1157/1157 - 322s - loss: 0.8190 - accuracy: 0.7221 - val_loss: 0.7229 - val_accuracy: 0.7569
Epoch 4/10
1157/1157 - 317s - loss: 0.6298 - accuracy: 0.7882 - val_loss: 0.6518 - val_accuracy: 0.7827
Epoch 5/10
1157/1157 - 261s - loss: 0.5684 - accuracy: 0.8135 - val_loss: 0.6571 - val_accuracy: 0.7752
Epoch 6/10
1157/1157 - 241s - loss: 0.4906 - accuracy: 0.8266 - val_loss: 0.5207 - val_accuracy: 0.8282
Epoch 7/10
1157/1157 - 216s - loss: 0.2952 - accuracy: 0.9079 - val_loss: 0.4645 - val_accuracy: 0.8657
Epoch 8/10
1157/1157 - 209s - loss: 0.1706 - accuracy: 0.9538 - val_loss: 0.4495 - val_accuracy: 0.8755
Epoch 9/10
1157/1157 - 205s - loss: 0.1137 - accuracy: 0.9715 - val_loss: 0.5109 - val_accuracy: 0.8726
Epoch 10/10
1157/1157 - 220s - loss: 0.0815 - accuracy: 0.9803 -